<font color='black' size='4'>0 Imports</font> <font color='red' size='3'></font>

In [1]:
%matplotlib inline
import os
import mne
import pandas as pd
from mne.externals.pymatreader import read_mat
import numpy as np
import matplotlib.pyplot as plt
import itertools
from glob import glob
import pyriemann
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import numpy as np

## 1 Dataset

Load the EEG recordings from shared drive or download form this link: https://zenodo.org/record/5055046#.YbNzGr3MJPb. 

This dataset has:
- 15 participants
- 3 session for each participants
- 3 levels of workload: Easy(0), Midium(1), Difficuilt(2)


<font color='black' size='4'>1.2 Set the Data Path</font> <font color='red' size='3'></font>

In [2]:
data_path = 'C:/Users/ursbu/Desktop/Seminar_Projekt_2022/Data'
n_subs = 15
n_sessions = 1
#diff = ['MATBeasy', 'MATBdiff']
#diff = ['MATBeasy', 'MATBmed']
diff = ['MATBeasy', 'MATBmed', 'MATBdiff']

<font color='black' size='4'>1.3 Read Electrodes channels names and its X,Y,Z position</font> <font color='red' size='3'></font>

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
channels_info = pd.read_csv(data_path + '/P01/S1/electrode_positions/get_chanlocs.csv', names=['ch_names','x','y','z'],sep =',') 
channels_info

,ch_names,x,y,z
0,Fp1,105.9276,29.3422,66.8159
1,Fz,66.4484,-2.1494,123.7135
2,F3,59.9094,45.3245,102.7517
3,F7,62.0175,68.6308,59.5730
4,F9,34.2379,78.5712,7.9506
5,FC5,34.9078,74.0046,77.9032
6,FC1,20.2263,39.8380,125.9352
7,C3,-12.4876,68.5622,104.8109
8,T7,-4.2346,86.0756,37.2070
9,ECG1,-53.2367,226.0020,-168.8785


## 2 Important Functions

<font color='black' size='4'>2.1 Bandpower Feature</font> <font color='red' size='3'></font>

In [4]:
#Raphael Vallat. (2022) UC Berkeley https://raphaelvallat.com/bandpower.html (Accessed: February 2022)

def bandpower(data, sf, band, window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    band = np.asarray(band)
    low, high = band

    # Define window length
    if window_sec is not None:
        nperseg = window_sec * sf
    else:
        nperseg = (2 / low) * sf

    # Compute the modified periodogram (Welch)
    freqs, psd = welch(data, sf, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

<font color='black' size='4'>2.2 Cross Validation</font> <font color='red' size='3'></font>

In [5]:
def cross_validation_eeg(clf,ch_slice,electrode_settings):
    acc_total=0.0
    for sub_n, session_n in itertools.product(range(n_subs), range(n_sessions)):
        epochs_data = []
        labels = []
        for lab_idx, level in enumerate(diff):
            sub = 'P{0:02d}'.format(sub_n+1)
            sess = f'S{session_n+1}'
            path =  os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
            
            # Read the epoched data with MNE
            epochs = mne.io.read_epochs_eeglab(path, verbose=False)
            
            # Channel selection based on the given Electrode Set(ch_slice)
            epochs = epochs.drop_channels(list(set(epochs.ch_names)-set(ch_slice)))

            # Get the data and concatenante with others MATB levels
            tmp = epochs.get_data()
            epochs_data.extend(tmp)
            labels.extend([lab_idx]*len(tmp))

        epochs_data = np.array(epochs_data)
        labels = np.array(labels)
        epochs_data , labels=shuffle(epochs_data,labels)
        
        # Compute classification accuracy with 5-folds cross validation
        acc = cross_val_score(clf, X=epochs_data, y=labels, cv=5)
        print(f'CV5_Score for {str(sub_n+1).zfill(2)}_sess{session_n+1} with'+electrode_settings+' settings: '+str(acc.mean()))
        acc_total+=acc.mean()
        print(acc_total)
        #print(str(labels))
        
    print('\n')
    print('Overall accuary over all '+str(sub_n+1)+' Participants and '+str(session_n+1)+' Session(s): '+str(acc_total/(n_subs)))

<font color='black' size='4'>2.3 CV and Testing on Bandpower feature</font> <font color='red' size='3'></font>

In [6]:
def testing_eeg(clf,ch_slice,electrode_settings):
    print('CV and Testing on Bandpower feature:')
    cv_score_total=0
    result_total=0
    for sub_n in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]:

        epochs_data = []
        labels = []
        participants = []
        for lab_idx, level in enumerate(diff):
            sub = 'P{0:02d}'.format(sub_n+1)
            sess = f'S{1}'
            path =  os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{1}_{level}.set'
            
            # Read the epoched data with MNE
            epochs = mne.io.read_epochs_eeglab(path, verbose=False)

            # Channel selection based on the given Electrode Set(ch_slice)
            epochs = epochs.drop_channels(list(set(epochs.ch_names)-set(ch_slice)))
            
            # Get the data and concatenante with others MATB levels
            tmp = epochs.get_data()
            win_sec = 2
            for i in range(len(tmp)):
                tmp_new = [list(map(lambda x: [bandpower(x, 500.00, [1, 4], win_sec, False),bandpower(x, 500.00, [4, 8], win_sec, False),bandpower(x, 500.00, [8, 12], win_sec, False),bandpower(x, 500.00, [12, 30], win_sec, False)], tmp[i]))]
                epochs_data.extend(tmp_new)
            labels.extend([lab_idx]*len(tmp))
            participants.extend([sub_n+1]*len(tmp))

        epochs_data = np.array(epochs_data)
        labels = np.array(labels)
        participants = np.array(participants)

        cv_score = cross_val_score(clf, X=epochs_data, y=labels, cv=5)
        cv_score = cv_score.mean()
        cv_score_total+=cv_score
        #print('\n')
        #print(f'CV5_Score for {str(sub_n+1).zfill(2)}_sess{1} with'+electrode_settings+' settings: '+str(cv_score.mean()))

        clf.fit(epochs_data,labels)

        n_subs = 15
        n_sessions = 2

        epochs_data_2 = []
        labels_2 = []
        participants_2 = []
        for lab_idx, level in enumerate(diff):
            sub = 'P{0:02d}'.format(sub_n+1)
            sess = f'S{2}'
            path =  os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{2}_{level}.set'
            # Read the epoched data with MNE
            epochs = mne.io.read_epochs_eeglab(path, verbose=False)
            
            # Channel selection based on the given Electrode Set(ch_slice)
            epochs = epochs.drop_channels(list(set(epochs.ch_names)-set(ch_slice)))
            #print(session_n+1)

            # Get the data apply Bandpower feature and concatenante with others MATB levels
            tmp = epochs.get_data()
            win_sec = 2
            for i in range(len(tmp)):
                tmp_new = [list(map(lambda x: [bandpower(x, 500.00, [1, 4], win_sec, False),bandpower(x, 500.00, [4, 8], win_sec, False),bandpower(x, 500.00, [8, 12], win_sec, False),bandpower(x, 500.00, [12, 30], win_sec, False)], tmp[i]))]

                epochs_data_2.extend(tmp_new)
            labels_2.extend([lab_idx]*len(tmp))
            participants_2.extend([sub_n+1]*len(tmp))

        epochs_data_2 = np.array(epochs_data_2)
        labels_2 = np.array(labels_2)
        participants_2 = np.array(participants_2)

        result=clf.score(epochs_data_2,labels_2)
        result_total+=result
        #print('Testing accuary for '+str(sub_n+1)+'. Participant on 2nd Session: '+str(result))
    print('\n')
    print('Average CV5_Score over all 15 Paricipants: '+str(cv_score_total/15))
    print('Average Testing Accuary over all 15 Paricipants: '+str(result_total/15))
    print('\n')
    print('\n')
    return clf

<font color='black' size='4'>2.3 CV and Testing on Pre-Processed EEG Data</font> <font color='red' size='3'></font>

In [7]:
def testing_raw_eeg(clf,ch_slice,electrode_settings):
    print('CV and Testing on pre-processed raw eeg data:')
    cv_score_total=0
    result_total=0
    for sub_n in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]:

        epochs_data = []
        labels = []
        participants = []
        for lab_idx, level in enumerate(diff):
            sub = 'P{0:02d}'.format(sub_n+1)
            sess = f'S{1}'
            path =  os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{1}_{level}.set'
            
            # Read the epoched data with MNE
            epochs = mne.io.read_epochs_eeglab(path, verbose=False)
            
            # Channel selection based on the given Electrode Set(ch_slice)
            epochs = epochs.drop_channels(list(set(epochs.ch_names)-set(ch_slice)))

            # Get the data and concatenante with others MATB levels
            tmp = epochs.get_data()
            epochs_data.extend(tmp)
            labels.extend([lab_idx]*len(tmp))
            participants.extend([sub_n+1]*len(tmp))
            
        epochs_data = np.array(epochs_data)
        labels = np.array(labels)
        participants = np.array(participants)

        cv_score = cross_val_score(clf, X=epochs_data, y=labels, cv=5)
        cv_score = cv_score.mean()
        cv_score_total+=cv_score
        #print('\n')
        #print(f'CV5_Score for {str(sub_n+1).zfill(2)}_sess{1} with'+electrode_settings+' settings: '+str(cv_score.mean()))
        clf.fit(epochs_data,labels)

        n_subs = 15
        n_sessions = 2

        epochs_data_2 = []
        labels_2 = []
        participants_2 = []
        for lab_idx, level in enumerate(diff):
            sub = 'P{0:02d}'.format(sub_n+1)
            sess = f'S{2}'
            path =  os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{2}_{level}.set'
            
            # Read the epoched data with MNE
            epochs = mne.io.read_epochs_eeglab(path, verbose=False)
            
            # Channel selection based on the given Electrode Set(ch_slice)
            epochs = epochs.drop_channels(list(set(epochs.ch_names)-set(ch_slice)))
            #print(session_n+1)

            # Get the data and concatenante with others MATB levels
            tmp = epochs.get_data()
            epochs_data_2.extend(tmp)
            labels_2.extend([lab_idx]*len(tmp))
            participants_2.extend([sub_n+1]*len(tmp))

        epochs_data_2 = np.array(epochs_data_2)
        labels_2 = np.array(labels_2)
        participants_2 = np.array(participants_2)

        result=clf.score(epochs_data_2,labels_2)
        result_total+=result
        #print('Testing accuary for '+str(sub_n+1)+'. Participant on 2nd Session: '+str(result))
    print('\n')
    print('Average CV5_Score over all 15 Paricipants: '+str(cv_score_total/15))
    print('Average Testing Accuary over all 15 Paricipants: '+str(result_total/15))
    print('\n')
    return clf

## 3.Cross-Validation and Testing with different Electrode Combination

<font color='black' size='3'>Minimalist Electrode Combination</font> <font color='red' size='3'>(2 Electrodes)</font>

In [8]:
import warnings
warnings.filterwarnings("ignore")

lr = LogisticRegression(C=1/10.,max_iter=1000)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
#ch_slice = ['AF3','AFz','AF4','AF8','F1','Fz','F2','F4','F6','F8','FC3','P8','POz','PO4','PO6','PO8','O1','OZ','O2','Iz']
#ch_slice = ['P8','POz','PO4','PO6','PO8','O1','OZ','O2','Iz']
ch_slice = ['Fz','POz']
electrode_settings=' minimal'

testing_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)

CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.4393474823137744
Average Testing Accuary over all 15 Paricipants: 0.4198359433258762




CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.5339209321681231
Average Testing Accuary over all 15 Paricipants: 0.4738255033557046




Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier',
                 TSclassifier(clf=LogisticRegression(C=0.1, max_iter=1000)))])

<font color='black' size='3'>[set0] Average Use Case</font> <font color='red' size='3'>(13 Electrodes)</font>

In [9]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'AF3', 'AFz', 'AF4','FP1', 'FP2', 'FPz']
electrode_settings=' Average_use_Case'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.7085027049521432
Average Testing Accuary over all 15 Paricipants: 0.3683818046234153


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.4815647107781939
Average Testing Accuary over all 15 Paricipants: 0.4301267710663684






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])

<font color='black' size='3'>[set1] All electrodes </font> <font color='red' size='3'>(62 Electrodes)</font>

In [10]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'CP6', 'CP2', 'FCz', 'C4', 'T8', 'FT8', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT10', 'F6', 'AF8', 'AF4', 'F2']
electrode_settings=' All Electrodes'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.9256462754889723
Average Testing Accuary over all 15 Paricipants: 0.4235645041014168


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.6024286308780692
Average Testing Accuary over all 15 Paricipants: 0.4504101416853094






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])

<font color='black' size='3'>[set2] Wavelet-power-Delta >=0.05 and <= -0.05 </font> <font color='red' size='3'>(30 Electrodes)</font>

In [20]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['AF3','AFz','AF4','AF8','F1','Fz','F2','F4','F6','F8','FC3','T7','C5','C3','C4','TP7','CP5','CP3','CP1','CPz','P7','P5','P3','P8','POz','PO4','PO8','O1','Oz','O2']
electrode_settings=' WpD>=0.05 and <=-0.05'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.8814831460674156
Average Testing Accuary over all 15 Paricipants: 0.37718120805369126


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.5765859342488555
Average Testing Accuary over all 15 Paricipants: 0.4366890380313199






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])

<font color='black' size='3'>[set3] Wavelet-power-Delta >=0.05 (Frontal electrodes only)</font> <font color='red' size='3'>(~ 30 sec) (11 Electrodes)</font>

In [12]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['AF3','AFz','AF4','AF8','F1','Fz','F2','F4','F6','F8','FC3']
electrode_settings=' WpD>=0.05 (Frontal electrodes only)'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.7336079900124842
Average Testing Accuary over all 15 Paricipants: 0.3698732289336316


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.49739159384103204
Average Testing Accuary over all 15 Paricipants: 0.41894108873974645






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])

<font color='black' size='3'>[set4] Wavelet-power-Delta >=0.05 (non-Frontal electrodes only)</font> <font color='red' size='3'>(~ 30 sec) (12 Electrodes)</font>

In [13]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['T7','C5','C3','C4','TP7','CP5','CP3','CP1','CPz','P7','P5','P3']
electrode_settings=' WpD>=0.05 (non-Frontal electrodes only)'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.7723778610070745
Average Testing Accuary over all 15 Paricipants: 0.33407904548844153


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.4985934248855598
Average Testing Accuary over all 15 Paricipants: 0.3907531692766592






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])

<font color='black' size='3'>[set5] Wavelet-power-Delta <= -0.05 (all non-Frontal)</font> <font color='red' size='3'>(~ 20 sec) (7 Electrodes)</font>

In [14]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['P8','POz','PO4','PO8','O1','Oz','O2']
electrode_settings=' WpD<= -0.05 (all non-Frontal)'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.6990362047440699
Average Testing Accuary over all 15 Paricipants: 0.4275913497390008


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.5020441115272576
Average Testing Accuary over all 15 Paricipants: 0.4301267710663684






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])

<font color='black' size='3'>[set6] Wavelet-power-Delta >=0.1</font> <font color='red' size='3'>(~ 30 sec) (14 Electrodes)</font>

In [15]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['AFz','Fz','F2','F4','F6','FC3','C5','C4','TP7','CP5','CP3','CPz','P7','P5']
electrode_settings=' WpD>=0.1'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.7882763212650853
Average Testing Accuary over all 15 Paricipants: 0.38374347501864287


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.51051518934665
Average Testing Accuary over all 15 Paricipants: 0.41372110365398956






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])

<font color='black' size='3'>[set7] Wavelet-power-Delta >=0.05</font> <font color='red' size='3'>(~ 45 sec) (23 Electrodes)</font>

In [16]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'),pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['AF3','AFz','AF4','AF8','F1','Fz','F2','F4','F6','F8','FC3','T7','C5','C3','C4','TP7','CP5','CP3','CP1','CPz','P7','P5','P3']
electrode_settings=' WpD>=0.05'

#cross_validation_eeg(clf,ch_slice,electrode_settings)
testing_raw_eeg(clf,ch_slice,electrode_settings)
testing_eeg(clf,ch_slice,electrode_settings)

CV and Testing on pre-processed raw eeg data:


Average CV5_Score over all 15 Paricipants: 0.8516987099459008
Average Testing Accuary over all 15 Paricipants: 0.3618195376584638


CV and Testing on Bandpower feature:


Average CV5_Score over all 15 Paricipants: 0.5409088639200998
Average Testing Accuary over all 15 Paricipants: 0.4199850857568978






Pipeline(steps=[('covariances', Covariances(estimator='oas')),
                ('tsclassifier', TSclassifier(clf=LogisticRegression(C=0.1)))])